## [9] 앙상블(ENSEMBLE) - Voting 방식
- 여러개의 모델 또는 동일 모델과 샘플링 데이터셋으로 병렬학습 진행하는 방식
- Voting 방식/ 기법
    - 구성 : 동일 데이터셋 + 학습 알고리즘이 다른 모델 
    - 결과 도출 : Hard(직접), Soft(간접)

- 유방암 판별 모델 구현 => 피쳐 30개 , 타겟 2개 (2진 분류)

In [42]:
# 모듈 로딩 
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np

## [1] 데이터 준비

In [43]:
data_set = load_breast_cancer(as_frame=True)
print(data_set.keys())

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])


In [44]:
# 데이터 확인 
print(f'target_names : {data_set["target_names"]}')
print(f'feature_names : {data_set["feature_names"]}')
print(f'DESCR \n {data_set["DESCR"]}')

target_names : ['malignant' 'benign']
feature_names : ['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
DESCR 
 .. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (sta

In [45]:
# 피쳐와 타겟 저장 
feature_df = data_set['data']
target_sr = data_set['target']

In [46]:
feature_df.head(2)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902


In [47]:
target_sr.head(2)

0    0
1    0
Name: target, dtype: int32

## [2] 훈련, 테스트 데이터 분리

In [48]:
from sklearn.model_selection import train_test_split

In [49]:
x_train, x_test, y_train, y_test = train_test_split(feature_df, target_sr, stratify=target_sr, random_state=10)

## [3] 학습 진행 => 앙상블의 보팅 방식 진행
- 데이터셋 동일
- 알고리즘 모델 : KNeighborsClassifier, LogisticRegression, DecisionTreeClassifier

In [50]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

In [51]:
# 알고리즘 인스턴스 생성 
k_model = KNeighborsClassifier()
dt_model = DecisionTreeClassifier( random_state= 10) # random_state : 튜닝 등 변경이 필요하다면 통합시켜야 함
lr_model = LogisticRegression(solver='liblinear') # liblinear 권장 : 일반적으로 속도가 빠름 

In [57]:
# 보팅 인스턴스 생성 
v_model = VotingClassifier(estimators=[('k_model', k_model), ('dt_model',dt_model) ,('lr_model',lr_model)], voting='hard') 
# default : voting = 'hard'

vs_model = VotingClassifier(estimators=[('k_model', k_model), ('dt_model',dt_model) ,('lr_model',lr_model)], voting='soft') 


In [60]:
# 학습 진행 
v_model.fit(x_train.values, y_train.values) # hard : 직접 선거 방식으로 (predict와 비슷)

vs_model.fit(x_train.values, y_train.values) # soft : 간접 선거 방식으로 모델마다의 확률값 합계에 대한 평균 ( predict_proba와 비슷한 것을 내서 계산 )

VotingClassifier(estimators=[('k_model', KNeighborsClassifier()),
                             ('dt_model',
                              DecisionTreeClassifier(random_state=10)),
                             ('lr_model',
                              LogisticRegression(solver='liblinear'))],
                 voting='soft')

In [61]:
# model parameter 확인 
print(f'[v_model.classes_] : {v_model.classes_}')
print(f'[v_model.estimators_] : {v_model.estimators_}')
print(f'[v_model.named_estimatios_] : {v_model.named_estimators_}')
print()

print(f'[v_model.n_features_in_] : {v_model.n_features_in_}')
print(f'[v_model.feature_names_in_] : {v_model.feature_names_in_}')

[v_model.classes_] : [0 1]
[v_model.estimators_] : [KNeighborsClassifier(), DecisionTreeClassifier(random_state=10), LogisticRegression(solver='liblinear')]
[v_model.named_estimatios_] : {'k_model': KNeighborsClassifier(), 'dt_model': DecisionTreeClassifier(random_state=10), 'lr_model': LogisticRegression(solver='liblinear')}

[v_model.n_features_in_] : 30
[v_model.feature_names_in_] : ['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']


## [4] 성능 확인 => train_ds, validation_ds 없어서 train_ds와 test_ds

In [55]:
train_score = v_model.score(x_train, y_train)

AttributeError: 'Flags' object has no attribute 'c_contiguous'

In [ ]:
v_model2 = VotingClassifier(estimators=[('k_model', k_model) ,('lr_model',lr_model)], voting='hard') 

In [ ]:
# c_contiguous 는 KNN에서 오류 발생 
v_model2.fit(x_train.values, y_train.values)

VotingClassifier(estimators=[('k_model', KNeighborsClassifier()),
                             ('lr_model',
                              LogisticRegression(solver='liblinear'))])

In [ ]:
# model parameter 확인 
print(f'[v_model2.classes_] : {v_model2.classes_}')
print(f'[v_model2.estimators_] : {v_model2.estimators_}')
print(f'[v_model2.named_estimatios_] : {v_model2.named_estimators_}')
print()

print(f'[v_model2.n_features_in_] : {v_model2.n_features_in_}')
# print(f'[v_model2.feature_names_in_] : {v_model2.feature_names_in_}') # fit() 할 때 DF 전달한 경우

[v_model2.classes_] : [0 1]
[v_model2.estimators_] : [KNeighborsClassifier(), LogisticRegression(solver='liblinear')]
[v_model2.named_estimatios_] : {'k_model': KNeighborsClassifier(), 'lr_model': LogisticRegression(solver='liblinear')}

[v_model2.n_features_in_] : 30


model 하나씩 빼면서 문제 찾기,,,,

=> ERROR MESSAGE : AttributeError : 'Flags' object has no attribute 'c_contiguous'

==> ndarray 타입으로 변경

-----------------

In [62]:
# 성능 확인 
train_score = v_model.score(x_train.values, y_train.values)
test_score = v_model.score(x_test.values, y_test.values)

soft_train_score = vs_model.score(x_train.values, y_train.values)
soft_test_score = vs_model.score(x_test.values, y_test.values)

print(f'[Hard] train_score : test_score = {train_score} : {test_score}')
print(f'[Soft] train_score : test_score = {soft_train_score} : {soft_test_score}')

[Hard] train_score : test_score = 0.9741784037558685 : 0.951048951048951
[Soft] train_score : test_score = 0.9882629107981221 : 0.951048951048951
